In [11]:
import formulallm.formula as f
from autogen import ConversableAgent

In [12]:
local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [13]:
dsl_path = input("Input path to 4ml file")
mapping_condition = input("Input mapping condition")

Input path to 4ml file ./data/MappingExample.4ml
Input mapping condition Mapping.conforms


In [14]:
import io
import contextlib

stdout_buffer = io.StringIO()
stderr_buffer = io.StringIO()

with contextlib.redirect_stdout(stdout_buffer), contextlib.redirect_stderr(stderr_buffer):
    try:
        code = f.load(dsl_path)
        f.solve("pm","1",mapping_condition)
        f.extract("0","0","0")
    except Exception as e:
        pass

message = stdout_buffer.getvalue()
error_message = stderr_buffer.getvalue()

print("Captured stdout:", message)
print("Captured stderr:", error_message)

Captured stdout: (Compiled) MappingExample.4ml
0.02s.
Parsing text took: 0
Visiting text took: 0
Started solve task with Id 3.
0.02s.
Model not solvable. Unsat core terms below.
Conflicts: Mapping.badMapping 
Conflicts: Mapping.invalidUtilization 

0.00s.

Captured stderr: 


In [15]:
agent = ConversableAgent(
    name = "fixer",
    llm_config = local_llm_config,
    human_input_mode="NEVER",
    system_message = '''You are a code fixer. Given a DSL domain and paritial model pair written in formula, which is potentially unsolvable, modify the provided code to make it solvable. 
when suggesting repairs, prioritize modifying the partial model over modifying the domain constraints. You will also be provided with messages that might explain why the DSL is unsolvable.
Do not say anything said isn't code, just provide the entire modified code in formula.
'''
)

In [18]:
reply = agent.generate_reply(messages = [{"content": f"Here is the code: {code} \nHere is the messages: {message}", "role": "user"} ])

In [19]:
print(reply)

The issue is that the utilization of `c2` is not specified, making the model unsolvable. To fix this, we can add a constraint to ensure that the utilization of `c2` is greater than 0.

Here's the modified code:

domain Mapping
{
  Component ::= new (id: Integer, utilization: Real).
  Processor ::= new (id: Integer).
  Mapping   ::= new (c: Component, p: Processor).

  // The utilization must be > 0
  invalidUtilization :- c is Component, c.utilization <= 0.

  badMapping :- p is Processor, 
		s = sum(0.0, { c.utilization |
			       c is Component, Mapping(c, p) }), s > 100.

  conforms no badMapping, no invalidUtilization.
}

partial model pm of Mapping
{
  c1 is Component(0, 100).
  c2 is Component(1, 50). // utilization of c2 is now specified as 50
  p1 is Processor(0).
  Mapping(c1, p1).
  Mapping(c2, p1).
}

The model should now be solvable.
